# AGiXT Enterprise Back End Tests

Initialize XT Systems SDK and run tests for the back end.
## Register a user

Register a user on the auth server.

In [ ]:
import random
import string
import time
import requests
from xts import XTSystemsSDK

def check_online(base_url: str, max_retries: int = 3, retry_delay: int = 10) -> bool:
    """
    Check if the API is online and responding
    Args:
        base_url: The base URL of the API
        max_retries: Maximum number of retries (default: 3)
        retry_delay: Delay between retries in seconds (default: 10)
    Returns:
        bool: True if API is online, False otherwise
    """
    retry_count = 0
    while retry_count < max_retries:
        try:
            response = requests.get(f"{base_url}/health", timeout=5)
            if response.status_code == 200:
                print(f"API is online at {base_url}")
                return True
        except (requests.RequestException, ConnectionError) as e:
            retry_count += 1
            if retry_count == max_retries:
                print(f"API is offline after {max_retries} attempts: {str(e)}")
                return False
            print(f"API check failed (attempt {retry_count}/{max_retries}), retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
    return False

# Initialize SDK
xts = XTSystemsSDK()

# Check if API is online before proceeding
if not check_online(xts.base_url):
    raise Exception("API is not available - aborting tests")

print("API is online - proceeding with tests")
time.sleep(60)

failures = 0
random_string = "".join(
    random.choices(string.ascii_uppercase + string.digits, k=10)
).lower()
test_email = f"{random_string}@xt.systems"

while failures < 100:
    try:
        otp_uri = xts.register_user(
            email=test_email, first_name="Test", last_name="User"
        )
        break
    except Exception as e:
        print(e)
        failures += 1
        time.sleep(5)

In [ ]:
# Show QR code for MFA setup
import qrcode
from IPython.display import Image

qr = qrcode.QRCode()
qr.add_data(otp_uri)
qr.make(fit=True)
img = qr.make_image(fill="black", back_color="white")
img.save("qr.png")
Image(filename="qr.png")

## Confirm user exists

In [ ]:
user_exists = xts.user_exists(email=test_email)

## Update User's Last Name

In [ ]:
update_user = xts.update_user(first_name="Super", last_name="Man")

## Get User Details

In [ ]:
user_details = xts.get_user()

import openai

openai.base_url = f"{xts.base_url}/v1/"
openai.api_key = xts.headers["Authorization"]

response = openai.chat.completions.create(
    model="XT",
    messages=[
        {
            "role": "user",
            "content": "How would I run the project?",
        }
    ],
    user="Time to test",
)

print(response.choices[0].message.content)

In [ ]:
conversation = xts.get_conversation(conversation_name="Time to test")
print(conversation)

conversations = xts.get_conversations()
print(conversations)

## Invite a User to a Company

In [ ]:
invitation = xts.create_invitation(
    email="invited_user@example.com",
    company_id="",
    role_id=3,  # Assuming 3 is the role_id for a regular user
)

## Accept Invitation

In [ ]:
accept_invitation = xts.accept_invitation(invitation_id=invitation["id"])

## Delete User

In [ ]:
delete_user_result = xts.delete_user()